In [ ]:
import pandas as pd
import itertools as it
from gurobipy import Model, GRB, quicksum

# Load and clean your data
data = pd.read_csv('/Users/cindychang/Documents/school/大二/OR/Mid/112-2-OR-MID/midterm/data/instance03.csv')

def parse_machine_list(machine_list):
    if pd.isna(machine_list):
        return []
    return list(map(int, machine_list.split(',')))

data['Stage-1 Machines'] = data['Stage-1 Machines'].apply(parse_machine_list)
data['Stage-2 Machines'] = data['Stage-2 Machines'].apply(parse_machine_list)

# set up all the machine's id
all_machines = set()
data['Stage-1 Machines'].apply(lambda machines: all_machines.update(machines))
data['Stage-2 Machines'].apply(lambda machines: all_machines.update(machines))

0    None
1    None
2    None
Name: Stage-2 Machines, dtype: object

In [ ]:
# Initialize the model
model = Model("Job_Scheduling")

# Extract jobs, machines, and assume stages and data are set
jobs = data['Job ID'].unique()
machines = set(sum(data['Stage-1 Machines'].tolist(), []) + sum(data['Stage-2 Machines'].tolist(), []))

# Define a large M
M = 10000000000

In [ ]:
c = model.addVars(jobs, [1, 2], name = "c", vtype = GRB.CONTINUOUS, lb = 0.0)  # Completion times
x = model.addVars(jobs, [1, 2], machines, name = "x", vtype = GRB.BINARY)      # Machine assignment
y = model.addVars(jobs, [1, 2], jobs, [1, 2], machines, name = "y", vtype = GRB.BINARY)
T = model.addVars(jobs, name = "T", vtype = GRB.CONTINUOUS, lb = 0.0)          # Tardiness

In [ ]:
# Objective function
model.setObjective(quicksum(T[j] for j in jobs), GRB.MINIMIZE)

In [ ]:
# 1. Job assignment for each stage
for j in jobs:
    # For stage 1:
    stage_1_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-1 Machines']
    # model.addConstr((x[j, 1, k] for k in machines if k not in stage_1_machines) == 0, name=f"Job_Assignment_Stage1_{j}_0")
    model.addConstr(quicksum(x[j, 1, k] for k in stage_1_machines) == 1, name=f"Job_Assignment_Stage1_{j}_1")
    model.addConstr(quicksum(x[j, 1, k] for k in machines) <= 1, name=f"Job_Assignment_Stage1_{j}_2")
    
    # For stage 2:
    stage_2_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-2 Machines']
    if stage_2_machines == []:
        model.addConstr(quicksum(x[j, 2, k] for k in machines) == 0, name=f"Job_Assignment_Stage2_{j}")
    else:
        # model.addConstr((x[j, 2, k] for k in machines if k not in stage_2_machines) == 0, name=f"Job_Assignment_Stage2_{j}_0")
        model.addConstr(quicksum(x[j, 2, k] for k in stage_2_machines) == 1, name=f"Job_Assignment_Stage2_{j}_1")
        model.addConstr(quicksum(x[j, 2, k] for k in machines) <= 1, name=f"Job_Assignment_Stage2_{j}_2")

In [ ]:
# 2. Interprocess time constraint
for j in jobs:
    p1 = data.loc[data['Job ID'] == j, 'Stage-1 Processing Time'].values[0]
    p2 = data.loc[data['Job ID'] == j, 'Stage-2 Processing Time'].values[0]
    print(f"Job {j}: {p1}, {p2}")
    model.addConstr(c[j, 2] - c[j, 1] - p2 <= 1)
    model.addConstr(c[j, 2] - c[j, 1] - p2 >= 0)
    if p2 == 0:
        model.addConstr(c[j, 2] == c[j, 1])

Job 1: 2.7, 1.3
Job 2: 1.6, 1.4
Job 3: 0.7, 1.9


In [ ]:
# 3. Job order constraint
for j, l in it.combinations(jobs, 2):
    # print(j, l);
    for i in machines:
        model.addConstr((x[j, 1, i] + x[l, 1, i] <= y[j, 1, l, 1, i] + y[l, 1, j, 1, i] + 1))
        model.addConstr((x[j, 1, i] + x[l, 2, i] <= y[j, 1, l, 2, i] + y[l, 2, j, 1, i] + 1))
        model.addConstr((x[j, 2, i] + x[l, 1, i] <= y[j, 2, l, 1, i] + y[l, 1, j, 2, i] + 1))
        model.addConstr((x[j, 2, i] + x[l, 2, i] <= y[j, 2, l, 2, i] + y[l, 2, j, 2, i] + 1))

for j, l in it.combinations(jobs, 2):
    p_l1 = data.loc[data['Job ID'] == l, 'Stage-1 Processing Time'].values[0]
    p_l2 = data.loc[data['Job ID'] == l, 'Stage-2 Processing Time'].values[0]
    # For j stage 1
    stage_1_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-1 Machines']
    for i in stage_1_machines:
        model.addConstr(c[j, 1] + p_l1 - c[l, 1] <= M*(1 - y[j, 1, l, 1, i]))
        model.addConstr(c[j, 1] + p_l2 - c[l, 2] <= M*(1 - y[j, 1, l, 2, i]))
    # For j stage 2
    stage_2_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-2 Machines']
    for i in stage_2_machines:
        model.addConstr(c[j, 2] + p_l1 - c[l, 1] <= M*(1 - y[j, 2, l, 1, i]))
        model.addConstr(c[j, 2] + p_l2 - c[l, 2] <= M*(1 - y[j, 2, l, 2, i]))
    
    p_j1 = data.loc[data['Job ID'] == j, 'Stage-1 Processing Time'].values[0]
    p_j2 = data.loc[data['Job ID'] == j, 'Stage-2 Processing Time'].values[0]
    # For l stage 1
    stage_1_machines = data.at[data.index[data['Job ID'] == l][0], 'Stage-1 Machines']
    for i in stage_1_machines:
        model.addConstr(c[l, 1] + p_j1 - c[j, 1] <= M*(1 - y[l, 1, j, 1, i]))
        model.addConstr(c[l, 1] + p_j2 - c[j, 2] <= M*(1 - y[l, 1, j, 2, i]))
    # For l stage 2
    stage_2_machines = data.at[data.index[data['Job ID'] == l][0], 'Stage-2 Machines']
    for i in stage_2_machines:
        model.addConstr(c[l, 2] + p_j1 - c[j, 1] <= M*(1 - y[l, 2, j, 1, i]))
        model.addConstr(c[l, 2] + p_j2 - c[j, 2] <= M*(1 - y[l, 2, j, 2, i]))

In [ ]:
# 5. Linearized tardiness calculation
for j in jobs:
    d = data.loc[data['Job ID'] == j, 'Due Time'].values[0]
    model.addConstr(T[j] >= c[j, 2] - d)
    model.addConstr(T[j] >= 0)

In [ ]:
# Optimize the model
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Model is infeasible; computing IIS")
    model.computeIIS()
    model.write("model.ilp")

if model.status == GRB.OPTIMAL:
    print(f"Total Tardiness: {model.objVal}")
    for j in jobs:
        print(f"Job {j} Tardiness: {T[j].X}")
        for stage in [1, 2]:
            print(f"Stage {stage} completion time: {c[j, stage].X}")

# for stage in range(1,2):
    for j in jobs:
        for k in machines:
            for stage in [1, 2]:
                if x[j, stage, k].X == 1:
                    print(f"Job {j} Stage {stage} in machine: {k}")

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 21.6.0 21G72)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 96 rows, 93 columns and 285 nonzeros
Model fingerprint: 0x6b1cc935
Variable types: 9 continuous, 84 integer (84 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+10]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 0.9000001
Presolve removed 19 rows and 38 columns
Presolve time: 0.00s
Presolved: 77 rows, 55 columns, 234 nonzeros
Variable types: 9 continuous, 46 integer (46 binary)

Root relaxation: objective 0.000000e+00, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | I